# Reconstructing the path

In this assignment you will use Spark to compute the shortest path between two vertices. In the video, you have learned how to compute the distances between a source vertex and all other vertices in a graph. Now, your task is to reconstruct the shortest path, that is a sequence of vertices connected by the edges.

- Dataset location: /data/twitter/twitter_sample_small.txt
- Format: user_id \t follower_id

You can start with the code described in "Starter..." (see the next self-reading).

Your task is to find the shortest path between vertices 12 and 34. In case of multiple shortest paths (that is, disjoint paths with the same length), any will suffice. Output format is the sequence of vertices, delimited by a comma, without spaces. For example, the path “12 -> 42 -> 34” should be printed as:

    12,42,34
    
Hint: before submitting, check your stopping criteria. In BFS, the search was exhaustive, and in this task your program may terminate earlier, thus saving some precious time.

The result on the sample dataset:

    12,422,53,52,107,20,23,274,34
    
If you want to deploy the environment on your own machine, please use [bigdatateam/spark-course1](https://hub.docker.com/r/bigdatateam/spark-course1/) Docker container.

In [1]:
#! /usr/bin/env python

from pyspark import SparkConf, SparkContext
import re

In [2]:
try:
    sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local").set("spark.cores.max", "16"))
except:
    pass

In [3]:
#this is old code provided by Coursera
def parse_edge(s):
  user, follower = s.split("\t")
  return (int(user), int(follower))

def step(item):
  prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
  return (next_v, prev_d + 1)

def complete(item):
  v, old_d, new_d = item[0], item[1][0], item[1][1]
  return (v, old_d if old_d is not None else new_d)

In [4]:
#this is old code provided by Coursera
n = 4  # 400 number of partitions
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

x = 12
d = 0
distances = sc.parallelize([(x, d)]).partitionBy(n)
while True:
    candidates = distances.join(forward_edges, n).map(step)
    print ("candidates = ", candidates.collect())
    print ('\n')
    new_distances = distances.fullOuterJoin(candidates, n).map(complete, True).persist()
    count = new_distances.filter(lambda i: i[1] == d + 1).count()
    if count > 0:
        d += 1
        distances = new_distances
        print ("d = ", d, "count = ", count, "distances = ", distances.collect())
        print ('\n')
    else:
        break

('candidates = ', [(126, 1), (380, 1), (422, 1), (648, 1)])


('d = ', 1, 'count = ', 4, 'distances = ', [(648, 1), (380, 1), (12, 0), (126, 1), (422, 1)])


('candidates = ', [(126, 1), (380, 1), (422, 1), (648, 1), (53, 2)])


('d = ', 2, 'count = ', 1, 'distances = ', [(648, 1), (380, 1), (12, 0), (53, 2), (126, 1), (422, 1)])


('candidates = ', [(126, 1), (380, 1), (422, 1), (648, 1), (31, 3), (52, 3), (57, 3), (150, 3), (187, 3), (292, 3), (652, 3), (53, 2)])


('d = ', 3, 'count = ', 7, 'distances = ', [(292, 3), (648, 1), (12, 0), (652, 3), (52, 3), (380, 1), (57, 3), (53, 2), (126, 1), (422, 1), (150, 3), (187, 3), (31, 3)])


('candidates = ', [(126, 1), (380, 1), (422, 1), (648, 1), (107, 4), (31, 3), (52, 3), (57, 3), (150, 3), (187, 3), (292, 3), (652, 3), (53, 2), (13, 4)])


('d = ', 4, 'count = ', 2, 'distances = ', [(292, 3), (648, 1), (12, 0), (652, 3), (52, 3), (380, 1), (57, 3), (53, 2), (13, 4), (126, 1), (422, 1), (150, 3), (107, 4), (187, 3), (31, 3)])


('candid

In [ ]:
#this is new code

In [5]:
n = 4  # 150 number of partitions
start, end = 12, 34

def parse_edge(s):
    """Parse raw data 'user\tfollower into a tuple'"""
    user, follower = s.split("\t")
    return (int(user), int(follower))

def step(item):
    # Add one move along the graph
    prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
    return (next_v, prev_d + [next_v])

edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()

In [6]:
# We want to find a path from 'start' to 'end' thus require forward edges
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

# Create a dataset composed of a tuple (current node, path).
# We will fill up the array with possible path
paths = sc.parallelize([(start, [start])]).partitionBy(n)

def found():
    return  paths.filter(lambda x: x[0] == end).count()

while not found():
    paths = paths.join(forward_edges, n).map(step)
    
paths.cache()

final_paths = (paths
               .filter(lambda value: value[0] == end)
               .map(lambda value: value[1])
              ).cache()

result = final_paths.take(1)[0]

print(','.join(map(str,result)))

12,422,53,52,107,20,23,274,34
